<a href="https://colab.research.google.com/github/chuckcurtis/glg-ack/blob/main/notebooks/1.0-cgc-GrailQA_With_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GrailQA Dataset Exploration w/ BERT

Adpated from https://towardsdatascience.com/multi-label-multi-class-text-classification-with-bert-transformer-and-keras-c6355eccb63a

In [29]:
!pip install transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast

from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.math import confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall
from tensorflow.keras.utils import to_categorical

import pandas as pd
from sklearn.model_selection import train_test_split

In [23]:
!pip install tensorflow_addons
from tensorflow_addons.metrics import F1Score

In [2]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'grail_qa')

     |████████████████████████████████| 264 kB 5.0 MB/s 
     |████████████████████████████████| 76 kB 4.1 MB/s 
     |████████████████████████████████| 118 kB 19.3 MB/s 
     |████████████████████████████████| 243 kB 19.6 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


Using custom data configuration default


Dataset grail_qa downloaded and prepared to /root/.cache/huggingface/datasets/grail_qa/default/0.0.0/2be99c6902e842f3ba87dd36fa96a2236206438ebb67c5e5464a36e4049fd3fb. Subsequent calls will reuse this data.


In [3]:
df_train = pd.DataFrame(dataset['train'])
df_valid = pd.DataFrame(dataset['validation'])
df_test = pd.DataFrame(dataset['test'])

In [4]:
df_train.head()

,qid,question,answer,function,num_node,num_edge,graph_query,sparql_query,domains,level,s_expression
0,2101535001000,oxybutynin chloride 5 extended release film co...,"{'answer_type': ['Entity', 'Entity'], 'answer_...",none,2,1,"{'nodes': {'nid': [0, 1], 'node_type': ['class...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[medicine],,(AND medicine.routed_drug (JOIN medicine.route...
1,2100954014000,the type single-sex school are in which instit...,"{'answer_type': ['Entity', 'Entity', 'Entity',...",none,2,1,"{'nodes': {'nid': [0, 1], 'node_type': ['class...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[education],,(AND education.educational_institution (JOIN e...
2,3206374001000,the leaders of the earliest established religi...,"{'answer_type': ['Entity'], 'answer_argument':...",argmin,3,2,"{'nodes': {'nid': [0, 1, 2], 'node_type': ['cl...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[religion],,(ARGMIN religion.religious_leadership_title (J...
3,2100735000000,"on 07/01/1970, which warship v1.1 was hit?","{'answer_type': ['Entity'], 'answer_argument':...",none,2,1,"{'nodes': {'nid': [0, 1], 'node_type': ['class...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[user.patrick.default_domain],,(AND user.patrick.default_domain.warship_v1_1 ...
4,2102770001000,what is the language regulator of basque?,"{'answer_type': ['Entity'], 'answer_argument':...",none,2,1,"{'nodes': {'nid': [0, 1], 'node_type': ['class...",PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,[language],,(AND language.language_regulator (JOIN languag...


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44337 entries, 0 to 44336
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   qid           44337 non-null  object
 1   question      44337 non-null  object
 2   answer        44337 non-null  object
 3   function      44337 non-null  object
 4   num_node      44337 non-null  int64 
 5   num_edge      44337 non-null  int64 
 6   graph_query   44337 non-null  object
 7   sparql_query  44337 non-null  object
 8   domains       44337 non-null  object
 9   level         44337 non-null  object
 10  s_expression  44337 non-null  object
dtypes: int64(2), object(9)
memory usage: 3.7+ MB


In [6]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6763 entries, 0 to 6762
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   qid           6763 non-null   object
 1   question      6763 non-null   object
 2   answer        6763 non-null   object
 3   function      6763 non-null   object
 4   num_node      6763 non-null   int64 
 5   num_edge      6763 non-null   int64 
 6   graph_query   6763 non-null   object
 7   sparql_query  6763 non-null   object
 8   domains       6763 non-null   object
 9   level         6763 non-null   object
 10  s_expression  6763 non-null   object
dtypes: int64(2), object(9)
memory usage: 581.3+ KB


In [7]:
df_test.head()

,qid,question,answer,function,num_node,num_edge,graph_query,sparql_query,domains,level,s_expression
0,test-0,flagstaff observatory houses what telescope?,"{'answer_type': [], 'answer_argument': [], 'en...",,-1,-1,"{'nodes': {'nid': [], 'node_type': [], 'id': [...",,[],,
1,test-1,which currency does s. africa formerly use?,"{'answer_type': [], 'answer_argument': [], 'en...",,-1,-1,"{'nodes': {'nid': [], 'node_type': [], 'id': [...",,[],,
2,test-2,which is the tv soundtrack of genre r&b that f...,"{'answer_type': [], 'answer_argument': [], 'en...",,-1,-1,"{'nodes': {'nid': [], 'node_type': [], 'id': [...",,[],,
3,test-3,which transit line includes 172a cherry street...,"{'answer_type': [], 'answer_argument': [], 'en...",,-1,-1,"{'nodes': {'nid': [], 'node_type': [], 'id': [...",,[],,
4,test-4,who uses british virgin islands?,"{'answer_type': [], 'answer_argument': [], 'en...",,-1,-1,"{'nodes': {'nid': [], 'node_type': [], 'id': [...",,[],,


The test partition has empty lists for the 'domains' column 🤔

In [8]:
df_train['label'] = pd.Categorical([domains[0] for domains in df_train['domains']])
df_valid['label'] = pd.Categorical([domains[0] for domains in df_valid['domains']])

df_train['numeric_label'] = df_train['label'].cat.codes
df_valid['numeric_label'] = df_valid['label'].cat.codes

In [9]:
len(df_train['label'].unique()) == len(df_train['label'].unique())

True

In [10]:
print(len(df_train['label'].unique()))
df_train['label'].value_counts()

76


music                          2305
book                           2237
fictional_universe             2231
medicine                       2013
computer                       1923
                               ... 
user.patrick.default_domain      19
periodicals                      16
type                             14
user.jonathanwlowe.location       9
comedy                            5
Name: label, Length: 76, dtype: int64

### Fetch BERT pre-trained encoder and tokenizer

In [11]:
#######################################
### --------- Setup BERT ---------- ###
# Name of the BERT model to use
model_name = 'bert-base-uncased'
# Max length of tokens
max_length = 100
# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
#######################################
### ------- Build the model ------- ###
# TF Keras documentation: https://www.tensorflow.org/api_docs/python/tf/keras/Model
# Load the MainLayer
bert = transformer_model.layers[0]
# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
# Then build your model output
label = Dense(units=len(df_train['numeric_label'].value_counts()),
              kernel_initializer=TruncatedNormal(
                  stddev=config.initializer_range
                  ), 
              name='label')(pooled_output)
outputs = {'label': label}
# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiClass')
# Take a look at the model
model.summary()

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "BERT_MultiClass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       TFBaseModelOutputWithPool 109482240 
_________________________________________________________________
pooled_output (Dropout)      (None, 768)               0         
_________________________________________________________________
label (Dense)                (None, 76)                58444     
Total params: 109,540,684
Trainable params: 109,540,684
Non-trainable params: 0
_________________________________________________________________


### Model Training

In [ ]:
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
    )

loss = {'label': CategoricalCrossentropy(from_logits = True)}
metric = {'label': CategoricalAccuracy('accuracy')}

model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

y_label = to_categorical(df_train['numeric_label'])

x = tokenizer(
    text=df_train['question'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=False,
    verbose=True)

history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'label': y_label},
    validation_split=0.2,
    batch_size=64,
    epochs=10
)

In [15]:
model.save('model.hdf5')

### Evalutation

Using the dev partition since the test partition is unlabeled.

In [14]:
test_y = to_categorical(df_valid['numeric_label'])
test_x = tokenizer(
    text=df_valid['question'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding="max_length",
    pad_to_max_length=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=False,
    verbose=True)

model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'label': test_y}
)

212/212 [==============================] - 28s 133ms/step - loss: 5.3697 - accuracy: 0.2252


### Verify Accuracy

In [40]:
preds = model.predict(x={'input_ids': test_x['input_ids']})

In [66]:
correct = 0
for pred, expected in zip(argmax(preds['label'], axis=1),
                          df_valid['numeric_label']):
  if pred == expected:
    correct += 1
print(f"Accuracy: {correct / len(df_valid)}")

Accuracy: 0.2251959189708709


### Confusion Matrix

In [50]:
from tensorflow import argmax
confusion_matrix(df_valid['numeric_label'],
                 argmax(preds['label'], axis=1),
                 num_classes=len(df_valid['label'].unique()))

<tf.Tensor: shape=(76, 76), dtype=int32, numpy=
array([[ 49,   0,   0, ...,   0,   0,   0],
       [  0,  42,   0, ...,   0,   0,   0],
       [  0,   0, 100, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,  17,   0,   0],
       [  0,   0,   0, ...,   0,  41,   0],
       [  0,   0,   0, ...,   0,   0,   7]], dtype=int32)>

###Precision, Recall, F1-Score

In [75]:
from tensorflow.math import count_nonzero, divide, multiply, add, subtract
predicted = preds['label']
actual = to_categorical(df_valid['numeric_label'])
TP = count_nonzero(predicted * actual)
TN = count_nonzero((predicted - 1) * (actual - 1))
FP = count_nonzero(predicted * (actual - 1))
FN = count_nonzero((predicted - 1) * actual)

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = (2 * precision * recall) / (precision + recall)
print(f"precision: {precision * 100}%")
print(f"recall: {recall * 100}%")
print(f"f1-score: {f1 * 100}%")

precision: 1.3157894736842104%
recall: 50.0%
f1-score: 2.564102564102564%
